In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL of the page to scrape
url = "https://fbref.com/en/matches/46d9048f/"

# Get the game ID from the URL
game_id = url.split("/")[-2]

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the Shots table on the page
shots_table = soup.find("div", {"id": "switcher_shots"})

# Create empty lists to store data and create the columns
data = []
headers = ['Game_ID', 'Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event', 'SCA 2 Player', 'Event']

# Extract table rows
rows = shots_table.find("tbody").find_all("tr")
for row in rows:
    row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
    data.append(row_data)

# Create a DataFrame from the shot data
shots_df = pd.DataFrame(data, columns=headers)

# Find the home and away teams and formations
team_headers = soup.find_all("th", {"colspan": "2"})
teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

# Extracting the home and away teams and formations
home_team = teams[0]
away_team = teams[1]
home_formation = formations[0]
away_formation = formations[1]

# Add team information to the DataFrame
shots_df['Home_Team'] = home_team
shots_df['Away_Team'] = away_team
shots_df['Home_Formation'] = home_formation
shots_df['Away_Formation'] = away_formation
shots_df['is_home_shot'] = np.where(shots_df['Team'] == home_team, 1, 0)
shots_df['is_away_shot'] = np.where(shots_df['Team'] == away_team, 1, 0)

In [13]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all links with match details excluding playoff games
match_links = soup.select("td[data-stat='match_report']:not(:has(a[href*='/playoffs/']) a)")

# Extract game IDs
game_IDs = [link["href"].split("/")[-2] for link in match_links]

# Print the game IDs
print(game_IDs)


KeyError: 'href'

In [9]:
len(game_IDs)


1114